In [107]:
import pandas as pd
import numpy as np
from pathlib import Path
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [108]:
try:
    project_root = Path.cwd().parent
    clean_data_path = project_root / "data" / "processed" / "credit_card_clients_clean.csv"
    df = pd.read_csv(clean_data_path)
    print(f"DataFrame limpio cargado desde: {clean_data_path}")
except FileNotFoundError:
    print(f"Error: No se encontró el archivo: {clean_data_path}")
    df = None


DataFrame limpio cargado desde: /Users/edusant/Desktop/personal/blue_tab/proyecto-riesgo-crediticio/data/processed/credit_card_clients_clean.csv


In [ ]:
# --- 1. Corrección de Fuga de Datos y Nueva Ingeniería de Características ---

df_features = df.copy()

# Definir grupos de columnas HISTÓRICAS para evitar data leakage
bill_amt_cols_hist = ['bill_amt_aug', 'bill_amt_july', 'bill_amt_june', 'bill_amt_may', 'bill_amt_april']
pay_amt_cols_hist = ['pay_amt_aug', 'pay_amt_july', 'pay_amt_june', 'pay_amt_may', 'pay_amt_april']
pay_status_cols_hist = ['pay_aug', 'pay_july', 'pay_june', 'pay_may', 'pay_april']

# -- A. Ratios de Salud Financiera --
epsilon = 1e-6
# Tasa de utilización del mes más reciente (Septiembre)
df_features['utilization_sept'] = df_features['bill_amt_sept'] / (df_features['limit_bal'] + epsilon)

# -- B. Agregados de Historial Financiero --
df_features['bill_amt_avg_hist'] = df_features[bill_amt_cols_hist].mean(axis=1)
df_features['bill_amt_std_hist'] = df_features[bill_amt_cols_hist].std(axis=1)
df_features['pay_amt_avg_hist'] = df_features[pay_amt_cols_hist].mean(axis=1)

# -- C. Características de Tendencia --
# Se calcula la pendiente sobre datos históricos para observar tendencias
def calculate_slope(row, cols):
    y = row[cols].values.astype(float)
    x = np.array(range(len(y)))
    slope, _ = np.polyfit(x, y, 1)
    return slope

# Aplicamos la función sobre las columnas históricas en orden cronológico (de más antiguo a más reciente)
df_features['bill_amt_slope'] = df_features.apply(lambda row: calculate_slope(row, bill_amt_cols_hist[::-1]), axis=1)
df_features['pay_status_slope'] = df_features.apply(lambda row: calculate_slope(row, pay_status_cols_hist[::-1]), axis=1)

# -- D. Indicadores de Comportamiento de Pago --
df_features['pay_status_avg_hist'] = df_features[pay_status_cols_hist].mean(axis=1)
df_features['months_with_delay_hist'] = (df_features[pay_status_cols_hist] > 0).sum(axis=1)

# Volatilidad de pagos: Un cliente con pagos muy irregulares puede ser riesgoso
all_pay_amt_cols = ['pay_amt_sept'] + pay_amt_cols_hist
df_features['payment_volatility'] = df_features[all_pay_amt_cols].std(axis=1) / (df_features[all_pay_amt_cols].mean(axis=1) + epsilon)

# Proxy de relación deuda-ingreso: Compara la deuda promedio con el límite total
df_features['debt_to_limit_ratio'] = df_features['bill_amt_avg_hist'] / (df_features['limit_bal'] + epsilon)

# Reemplazar valores infinitos o NaN que puedan surgir de las divisiones
df_features.replace([np.inf, -np.inf], np.nan, inplace=True)
df_features.fillna(0, inplace=True) 

print("--- Ingeniería de Características Mejorada (sin fuga de datos) ---")
display(df_features.head())

--- Ingeniería de Características Mejorada (sin fuga de datos) ---


,ID,limit_bal,sex,education,marriage,age,pay_sept,pay_aug,pay_july,pay_june,...,utilization_sept,bill_amt_avg_hist,bill_amt_std_hist,pay_amt_avg_hist,bill_amt_slope,pay_status_slope,pay_status_avg_hist,months_with_delay_hist,payment_volatility,debt_to_limit_ratio
0,1,20000,2,2,1,24,2,2,-1,-1,...,0.195650,758.2,1343.762330,137.8,689.3,9.000000e-01,-0.8,1,2.449490,0.037910
1,2,120000,2,2,2,26,-1,2,0,0,...,0.022350,2879.0,707.579324,1000.0,-384.5,-1.009748e-16,0.8,2,0.903327,0.023992
2,3,90000,2,2,2,34,0,0,0,0,...,0.324878,14482.8,780.608224,1900.0,-443.3,0.000000e+00,0.0,0,0.854864,0.160920
3,4,50000,2,2,1,37,0,0,0,0,...,0.939800,36868.8,10872.147773,1277.6,5770.4,0.000000e+00,0.0,0,0.341959,0.737376
4,5,50000,1,2,1,57,-1,0,-1,0,...,0.172340,20144.4,10704.671237,11409.8,-1023.3,-1.000000e-01,-0.2,0,1.400826,0.402888


In [110]:
if df is not None:
    # Se seleccionan solo las columnas numéricas para el cálculo de VIF.
    numeric_cols_for_vif = df_features.select_dtypes(include=np.number).drop(columns=['ID', 'default_payment_next_month'])
    
    # Se añade una constante para el cálculo correcto del VIF.
    X_vif = add_constant(numeric_cols_for_vif)
    
    vif_data = pd.DataFrame()
    vif_data["feature"] = X_vif.columns
    vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])]
    
    print("\n--- Análisis de Factor de Inflación de la Varianza (VIF) ---")
    # Se muestran las 15 variables con mayor VIF, que son las más problemáticas.
    display(vif_data.sort_values(by='VIF', ascending=False).head(15))

/opt/anaconda3/envs/riesgo_crediticio_env/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)



--- Análisis de Factor de Inflación de la Varianza (VIF) ---


,feature,VIF
9,pay_june,inf
8,pay_july,inf
29,pay_status_slope,inf
30,pay_status_avg_hist,inf
7,pay_aug,inf
16,bill_amt_may,inf
14,bill_amt_july,inf
13,bill_amt_aug,inf
11,pay_april,inf
10,pay_may,inf


In [113]:
if df is not None:
    features_reducidas = [
        'ID',
        # Demográficas y Límite de Crédito
        'limit_bal', 'sex', 'education', 'marriage', 'age',
        # Comportamiento de Pago (las más predictivas)
        'pay_sept', 'pay_status_slope',
        # Salud Financiera (menos correlacionadas)
        'utilization_sept', 'bill_amt_std_hist', 'pay_amt_avg_hist',
        # Variable Objetivo
        'default_payment_next_month'
    ]
    df_reducido = df_features[features_reducidas]
    print("\n--- Dataset Reducido Creado ---")
    display(df_reducido.head())



--- Dataset Reducido Creado ---


,ID,limit_bal,sex,education,marriage,age,pay_sept,pay_status_slope,utilization_sept,bill_amt_std_hist,pay_amt_avg_hist,default_payment_next_month
0,1,20000,2,2,1,24,2,9.000000e-01,0.195650,1343.762330,137.8,1
1,2,120000,2,2,2,26,-1,-1.009748e-16,0.022350,707.579324,1000.0,1
2,3,90000,2,2,2,34,0,0.000000e+00,0.324878,780.608224,1900.0,0
3,4,50000,2,2,1,37,0,0.000000e+00,0.939800,10872.147773,1277.6,0
4,5,50000,1,2,1,57,-1,-1.000000e-01,0.172340,10704.671237,11409.8,0


In [114]:
if df is not None:
    df_features_reg = df.copy()
    
    # Se usan solo datos hasta junio para evitar fuga de información.
    bill_cols_reg = ['bill_amt_june', 'bill_amt_may', 'bill_amt_april']
    pay_cols_reg = ['pay_amt_may', 'pay_amt_april']
    
    # 6.1. Ratios y Agregados
    df_features_reg['utilization_june'] = df_features_reg['bill_amt_june'] / (df_features_reg['limit_bal'] + epsilon)
    df_features_reg['bill_amt_avg_3m'] = df_features_reg[bill_cols_reg].mean(axis=1)
    df_features_reg['pay_amt_avg_2m'] = df_features_reg[pay_cols_reg].mean(axis=1)

    print("\n--- Ingeniería de Características para Regresión completada ---")



--- Ingeniería de Características para Regresión completada ---


In [115]:
if df is not None:
    try:
        processed_data_path = project_root / "data" / "processed"
        
        # Guardar dataset completo para clasificación
        path_clasificacion = processed_data_path / "features_clasificacion.csv"
        df_features.to_csv(path_clasificacion, index=False)
        print(f"\nDataset para clasificación guardado en: {path_clasificacion}")

        # Guardar dataset reducido para clasificación
        path_reducido = processed_data_path / "features_reducido_clasificacion.csv"
        df_reducido.to_csv(path_reducido, index=False)
        print(f"Dataset reducido para clasificación guardado en: {path_reducido}")
        
        # Guardar dataset para regresión
        path_regresion = processed_data_path / "features_regresion.csv"
        df_features_reg.to_csv(path_regresion, index=False)
        print(f"Dataset para regresión guardado en: {path_regresion}")

    except Exception as e:
        print(f"\nOcurrió un error al guardar los archivos: {e}")



Dataset para clasificación guardado en: /Users/edusant/Desktop/personal/blue_tab/proyecto-riesgo-crediticio/data/processed/features_clasificacion.csv
Dataset reducido para clasificación guardado en: /Users/edusant/Desktop/personal/blue_tab/proyecto-riesgo-crediticio/data/processed/features_reducido_clasificacion.csv
Dataset para regresión guardado en: /Users/edusant/Desktop/personal/blue_tab/proyecto-riesgo-crediticio/data/processed/features_regresion.csv
